In [1]:
import wandb

wandb.init(project="advanced_ai_imdb_dataset", name="alberta-test-run")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joatmon. Use `wandb login --relogin` to force relogin


In [2]:
from datasets import load_dataset

format_name = "format0"
# format_name = "format1"
# train_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/train.jsonl", split="train"
# )
# val_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/dev.jsonl", split="train"
# )
# test_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/test.jsonl", split="train"
# )
dataset = load_dataset(
    "json",
    data_files={
        "train": f"./vlsp_preprocessed/{format_name}/train.jsonl",
        "val": f"./vlsp_preprocessed/{format_name}/dev.jsonl",
        "test": f"./vlsp_preprocessed/{format_name}/test.jsonl",
    },
)

c:\Users\Asus\miniconda3\envs\ai_code\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2")

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def preprocess_token(example):
    return tokenizer(
        example["sentence"], padding="max_length", truncation=True, return_tensors="pt"
    )

In [5]:
tokenized_dataset = dataset.map(preprocess_token, batched=True)

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Map: 100%|██████████| 599/599 [00:00<00:00, 1961.59 examples/s]


In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [8]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels),
        "f1": f1.compute(predictions=predictions, references=labels),
        "precision": precision.compute(predictions=predictions, references=labels),
        "recall": recall.compute(predictions=predictions, references=labels),
    }

In [9]:
id2label = {0: 'AFFILIATION', 1: 'PART – WHOLE', 2: 'LOCATED', 3: 'PERSONAL - SOCIAL'}
label2id = {'AFFILIATION': 0, 'PART – WHOLE': 1, 'LOCATED': 2, 'PERSONAL - SOCIAL': 3}

In [10]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="albert_imdb",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    eval_steps=500,
    save_total_limit=1,
    # report_to="wandb",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [11]:
tokenized_dataset["train"][0]

{'sentence': '<location1/> <sep> <location2/> <sep> Trong ảnh : Nghệ_thuật tạo hoa_văn trên trang_phục truyền_thống của người Mông hoa , tại <location2/> , <location1/> .',
 'label': 2,
 'input_ids': [2,
  13,
  1,
  19032,
  8197,
  1,
  13,
  1,
  18,
  3492,
  1,
  13,
  1,
  19032,
  9298,
  1,
  13,
  1,
  18,
  3492,
  1,
  13,
  38,
  14271,
  40,
  252,
  13,
  45,
  13,
  2723,
  438,
  1,
  38,
  7325,
  38,
  14341,
  20538,
  1,
  2686,
  26407,
  13,
  38,
  13119,
  1,
  3971,
  6335,
  8600,
  8944,
  1,
  96,
  3279,
  13,
  22936,
  13,
  2723,
  13324,
  49,
  21028,
  20538,
  13,
  15,
  5466,
  13,
  1,
  19032,
  9298,
  1,
  13,
  15,
  13,
  1,
  19032,
  8197,
  1,
  13,
  9,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [12]:
trainer.train()

  0%|          | 0/1250 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
wandb.finish()